In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import os
import pandas as pd
from tqdm import tqdm
from pyade.sade import apply as sade
from pyade.jade import apply as jade
from pyade.de import apply as de
from pyade.lshade import apply as lshade
from fitnessFunctions import spherical, rosenbrock, elliptic, rastrigin, ackley

In [2]:
def getSequence(population, fitfunc):
    tmpPopulation = sorted(population, key=lambda point: fitfunc(point))
    centroidSequence = []
    while len(tmpPopulation):
        centroidSequence.append(np.mean(tmpPopulation, axis=0))
        del tmpPopulation[-1]
    return np.array(centroidSequence)


def estimator(population, fitfunc):
    samples = np.linspace(0, 1, len(population)).reshape(-1, 1)
    targets = getSequence(population, fitfunc)
    regression_model = LinearRegression().fit(samples, targets)
    return regression_model.predict([[1.0]])[0]

In [3]:
def desc_points(population, fitfunc):
    bf = min(map(lambda point: fitfunc(point), population))
    mf = fitfunc(np.mean(population, axis=0))
    ep = fitfunc(estimator(population, fitfunc))
    return bf, mf, ep

In [1]:
test_params_DN = [(10, 2), (10, 5), (10, 10), (10, 20), (10, 50), (10, 100), (10, 200), (10, 500), (10, 1000), (2, 100),
                  (5, 100), (10, 100), (20, 100), (50, 100), (100, 100)]

out_dir_base = '/Users/wojciechsitek/Documents/PW/ea-dotty/out/sade.nosync/'
fitnessFunctions = [spherical, rosenbrock, elliptic, rastrigin, ackley]
algorithms = [de, jade, sade, lshade]
algorithm_names = ['de', 'jade', 'sade', 'lshade']


with tqdm(total=len(fitnessFunctions) * len(test_params_DN) * len(algorithms)) as bar:
    for fitfunc in fitnessFunctions:
        for D, N in test_params_DN:
            for algorithm, algo_name in zip(algorithms, algorithm_names):
                params = {'max_evals': 5000 * N, 'population_size': N, 'callback': None,
                          'individual_size': D, 'seed': 21, 'opts': None,
                          'bounds': np.array([[-pow(10, 6), pow(10, 6)]] * D), 'func': fitfunc}
                if algo_name == 'jade':
                    params['c'] = 0.1
                    params['p'] = max(.05, 3 / N)
                elif algo_name == 'de':
                    params['f'] = 0.5
                    params['cr'] = 0.9
                    params['cross'] = 'bin'
                _, _, archive = algorithm(**params)
                descriptive_points = list(map(lambda population: desc_points(population, fitfunc), archive))
                path = os.path.join(out_dir_base, f"{algo_name}-{fitfunc.__name__}-D={D}-N={N}")
                pd.DataFrame(np.array(descriptive_points)).to_csv(path, index=False, header=False)
                bar.update(1)